<a href="https://colab.research.google.com/github/financieras/big_data/blob/main/leccion_2_1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lección 2.1.4: Relaciones entre variables (correlación, covarianza y tablas cruzadas)

## 1. El arte de conectar los puntos: Descubriendo relaciones ocultas

Analizar relaciones entre variables es como ser un **detective de conexiones**: encuentras hilos invisibles que unen diferentes aspectos de tus datos. No se trata solo de números—se trata de entender cómo el movimiento en una variable afecta a otras, revelando el tejido interdependiente de tu negocio.

> **Idea clave:** Las relaciones te dicen "cuando esto cambia, qué más cambia con ello"—el fundamento de la predicción y la comprensión.

**¿Por qué estas relaciones importan?**
- 🔮 **Predicción:** Si sabes cómo se relacionan las variables, puedes predecir comportamientos
- 🎯 **Eficiencia:** Enfocar esfuerzos en las variables que más impacto tienen
- 👥 **Segmentación:** Encontrar grupos naturales basados en relaciones compartidas
- 💡 **Insights de negocio:** Descubrir palancas para intervenir

**Ejemplo revelador:** Un retailer descubre que la **relación entre tiempo en website y tasa de conversión** no es lineal—hay un punto óptimo después del cual los usuarios se abruman. Esto transforma su estrategia de UX completamente.

**Ejemplo de Amazon:** Descubrieron que el tiempo de entrega correlaciona fuertemente con satisfacción (r=0.72), pero el empaque bonito NO (r=0.08). **Decisión:** Invertir millones en logística, no en diseño de cajas.

> **Advertencia crítica:** Correlación ≠ Causalidad. Siempre. Sin excepciones. El helado y los ahogamientos correlacionan (ambos suben en verano), pero el helado no causa ahogamientos.

---

## 2. Tipos de relaciones entre variables

### **Clasificación por tipo de variables**

| Relación | Variable 1 | Variable 2 | Métrica principal | Visualización |
|----------|-----------|-----------|-------------------|---------------|
| **Numérica-Numérica** | Continua/Discreta | Continua/Discreta | Correlación (Pearson) | Scatter plot |
| **Categórica-Categórica** | Nominal/Ordinal | Nominal/Ordinal | Chi-cuadrado, Cramér's V | Tabla cruzada, heatmap |
| **Numérica-Categórica** | Continua/Discreta | Nominal/Ordinal | ANOVA, Kruskal-Wallis | Boxplot agrupado |

### **Clasificación por dirección y fuerza**

| Tipo | Descripción | Ejemplo | Coeficiente |
|------|-------------|---------|-------------|
| **Positiva fuerte** | Ambas aumentan juntas | Altura vs peso | r > 0.7 |
| **Negativa fuerte** | Una sube, otra baja | Precio vs demanda | r < -0.7 |
| **Nula** | No hay relación | Número de zapato vs CI | r ≈ 0 |

---

## 3. Correlación: La danza sincronizada de las variables

### **Interpretación detallada de coeficientes**

| Valor | Fuerza | Dirección | Interpretación práctica |
|-------|--------|-----------|------------------------|
| **0.9 a 1.0** | Muy fuerte | Positiva | "Casi perfectamente sincronizadas" |
| **0.7 a 0.9** | Fuerte | Positiva | "Fuertemente relacionadas" |
| **0.5 a 0.7** | Moderada | Positiva | "Relación noticeable" |
| **0.3 a 0.5** | Débil | Positiva | "Relación leve" |
| **0.0 a 0.3** | Muy débil | Positiva | "Prácticamente no relacionadas" |
| **-0.3 a 0.0** | Muy débil | Negativa | "Prácticamente no relacionadas" |
| **-0.5 a -0.3** | Débil | Negativa | "Relación leve inversa" |
| **-0.7 a -0.5** | Moderada | Negativa | "Relación inversa noticeable" |
| **-0.9 a -0.7** | Fuerte | Negativa | "Fuertemente inversas" |
| **-1.0 a -0.9** | Muy fuerte | Negativa | "Casi perfectamente opuestas" |

### **Matriz de correlación completa**

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

def matriz_correlacion_completa(df):
    """Calcula y visualiza correlaciones entre todas las variables numéricas"""
    # Seleccionar solo columnas numéricas
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Matriz de correlación
    corr_matrix = numeric_df.corr()
    
    # Visualización con heatmap (solo triángulo inferior)
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))  # Máscara para triángulo superior
    sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm',
                center=0, square=True, fmt='.2f',
                linewidths=1, cbar_kws={"shrink": .8})
    plt.title('Matriz de Correlación (Triángulo Inferior)')
    plt.tight_layout()
    plt.show()
    
    return corr_matrix

# Uso
correlaciones = matriz_correlacion_completa(df)
```

### **Análisis de correlación pareada completo**

```python
def analizar_correlacion_pareada(df, var1, var2, alpha=0.05):
    """Análisis detallado de correlación entre dos variables"""
    # Limpiar datos
    data_clean = df[[var1, var2]].dropna()
    x = data_clean[var1]
    y = data_clean[var2]
    
    # Correlación de Pearson
    corr_pearson, p_value_pearson = stats.pearsonr(x, y)
    
    # Correlación de Spearman (robusta a outliers)
    corr_spearman, p_value_spearman = stats.spearmanr(x, y)
    
    # Correlación de Kendall (para muestras pequeñas)
    corr_kendall, p_value_kendall = stats.kendalltau(x, y)
    
    # Resultados
    print(f"\n=== ANÁLISIS DE CORRELACIÓN: {var1} vs {var2} ===")
    print(f"Pearson:  r = {corr_pearson:.3f}, p-value = {p_value_pearson:.4f}")
    print(f"Spearman: ρ = {corr_spearman:.3f}, p-value = {p_value_spearman:.4f}")
    print(f"Kendall:  τ = {corr_kendall:.3f}, p-value = {p_value_kendall:.4f}")
    
    # Interpretación de significancia
    if p_value_pearson < alpha:
        print(f"\n✅ La correlación es SIGNIFICATIVA (α = {alpha})")
    else:
        print(f"\n❌ La correlación NO es significativa (α = {alpha})")
    
    # Interpretación de fuerza
    if abs(corr_pearson) >= 0.7:
        fuerza = "fuerte"
    elif abs(corr_pearson) >= 0.5:
        fuerza = "moderada"
    elif abs(corr_pearson) >= 0.3:
        fuerza = "débil"
    else:
        fuerza = "muy débil"
    print(f"Fuerza de la relación: {fuerza}")
    
    # Visualización
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Scatter plot con regresión
    axes[0].scatter(x, y, alpha=0.6, s=50)
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    axes[0].plot(x, p(x), "r--", alpha=0.8, linewidth=2)
    axes[0].set_xlabel(var1)
    axes[0].set_ylabel(var2)
    axes[0].set_title(f'Scatter Plot\nPearson r = {corr_pearson:.3f}')
    axes[0].grid(True, alpha=0.3)
    
    # Jointplot simplificado (distribuciones marginales)
    axes[1].scatter(x, y, alpha=0.6, s=50)
    axes[1].set_xlabel(var1)
    axes[1].set_ylabel(var2)
    axes[1].set_title(f'p-value = {p_value_pearson:.4f}')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'pearson': {'r': corr_pearson, 'p_value': p_value_pearson},
        'spearman': {'rho': corr_spearman, 'p_value': p_value_spearman},
        'kendall': {'tau': corr_kendall, 'p_value': p_value_kendall}
    }

# Uso práctico
resultado = analizar_correlacion_pareada(df, 'precio', 'ventas')
```

**Cuándo usar cada método:**
- **Pearson:** Relación lineal, datos normales, sin outliers
- **Spearman:** Relación monótona (no necesariamente lineal), datos ordinales, con outliers
- **Kendall:** Muestras pequeñas (n<50), muchos empates, interpretación más intuitiva

---

## 4. Covarianza: Hermana de la correlación

### **Diferencia entre covarianza y correlación**

```python
def comparar_covarianza_correlacion(df, var1, var2):
    """Muestra la diferencia entre covarianza y correlación"""
    # Datos limpios
    data_clean = df[[var1, var2]].dropna()
    x = data_clean[var1]
    y = data_clean[var2]
    
    # Cálculos
    covarianza = np.cov(x, y)[0, 1]
    correlacion = np.corrcoef(x, y)[0, 1]
    
    print(f"\n=== COVARIANZA vs CORRELACIÓN ===")
    print(f"Covarianza: {covarianza:,.2f}")
    print(f"Correlación: {correlacion:.3f}")
    
    print(f"\n📊 DIFERENCIAS CLAVE:")
    print(f"• Covarianza: Mide dirección + magnitud (depende de escalas)")
    print(f"• Correlación: Mide dirección + fuerza (normalizada -1 a 1)")
    print(f"• Covarianza: Difícil de interpretar por sí sola")
    print(f"• Correlación: Fácil de interpretar y comparar")
    
    # Relación matemática
    std_x = x.std()
    std_y = y.std()
    print(f"\n📐 RELACIÓN MATEMÁTICA:")
    print(f"Correlación = Covarianza / (σx × σy)")
    print(f"{correlacion:.3f} = {covarianza:.2f} / ({std_x:.2f} × {std_y:.2f})")
    
    return covarianza, correlacion

# Ejemplo
cov, corr = comparar_covarianza_correlacion(df, 'precio', 'ventas')
```

| Aspecto | Covarianza | Correlación |
|---------|-----------|-------------|
| **Rango** | -∞ a +∞ | -1 a +1 |
| **Interpretación** | Difícil (depende de escalas) | Fácil (estandarizada) |
| **Sensibilidad a escala** | SÍ (cambia con unidades) | NO (normalizada) |
| **Uso común** | Matemática interna | Análisis e interpretación |

> **Consejo práctico:** Usa **correlación** para interpretar, usa **covarianza** solo si necesitas la matemática cruda (ej: álgebra lineal, finanzas).

---

## 5. Tablas cruzadas: Para variables categóricas

### **Análisis completo de relaciones categóricas**

```python
def analizar_relaciones_categoricas(df, cat1, cat2, normalizar=False):
    """Crea y analiza tablas cruzadas entre variables categóricas"""
    # Tabla cruzada básica
    tabla_cruzada = pd.crosstab(df[cat1], df[cat2])
    
    print(f"\n=== TABLA CRUZADA: {cat1} vs {cat2} ===")
    print("Frecuencias absolutas:")
    print(tabla_cruzada)
    print(f"\nTotal de observaciones: {tabla_cruzada.sum().sum()}")
    
    # Estadística Chi-cuadrado
    chi2, p_value, dof, expected = stats.chi2_contingency(tabla_cruzada)
    
    print(f"\n=== PRUEBA CHI-CUADRADO ===")
    print(f"Chi-cuadrado: {chi2:.3f}")
    print(f"P-value: {p_value:.4f}")
    print(f"Grados de libertad: {dof}")
    
    if p_value < 0.05:
        print("✅ Hay una relación SIGNIFICATIVA entre las variables")
    else:
        print("❌ NO hay evidencia de relación significativa")
    
    # Cramér's V (fuerza de asociación)
    n = tabla_cruzada.sum().sum()
    cramers_v = np.sqrt(chi2 / (n * (min(tabla_cruzada.shape) - 1)))
    
    if cramers_v < 0.1:
        fuerza = "muy débil"
    elif cramers_v < 0.3:
        fuerza = "débil"
    elif cramers_v < 0.5:
        fuerza = "moderada"
    else:
        fuerza = "fuerte"
    
    print(f"Cramér's V: {cramers_v:.3f} (asociación {fuerza})")
    
    # Tablas normalizadas
    if normalizar:
        print(f"\n=== TABLAS NORMALIZADAS ===")
        print("\nPor filas (%):")
        print((pd.crosstab(df[cat1], df[cat2], normalize='index') * 100).round(1))
        print("\nPor columnas (%):")
        print((pd.crosstab(df[cat1], df[cat2], normalize='columns') * 100).round(1))
    
    # Visualización
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Heatmap de frecuencias
    sns.heatmap(tabla_cruzada, annot=True, fmt='d', cmap='Blues', ax=axes[0])
    axes[0].set_title(f'Frecuencias Absolutas\n(Chi² = {chi2:.1f}, p = {p_value:.4f})')
    
    # Barras apiladas
    tabla_porcentaje = pd.crosstab(df[cat1], df[cat2], normalize='index') * 100
    tabla_porcentaje.plot(kind='bar', stacked=True, ax=axes[1])
    axes[1].set_title(f'Distribución Porcentual por {cat1}')
    axes[1].set_ylabel('Porcentaje')
    axes[1].legend(title=cat2, bbox_to_anchor=(1.05, 1))
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return tabla_cruzada, chi2, p_value, cramers_v

# Uso: Relación entre canal de adquisición y conversión
tabla, chi2, p_val, cramers = analizar_relaciones_categoricas(
    df, 'canal_adquisicion', 'convertido', normalizar=True
)
```

---

## 6. Relaciones mixtas: Numérica vs Categórica

### **Análisis con ANOVA**

```python
def analizar_relacion_mixta(df, cat_var, num_var):
    """Analiza relación entre variable categórica y numérica"""
    print(f"\n=== ANÁLISIS MIXTO: {cat_var} vs {num_var} ===")
    
    # Estadísticas por grupo
    stats_grupos = df.groupby(cat_var)[num_var].agg([
        'mean', 'std', 'count', 'median', 'min', 'max'
    ])
    print("\nEstadísticas por grupo:")
    print(stats_grupos)
    
    # ANOVA para significancia
    grupos = [group[1].dropna().values for group in df.groupby(cat_var)[num_var]]
    f_stat, p_value = stats.f_oneway(*grupos)
    
    print(f"\n=== ANOVA (One-Way) ===")
    print(f"F-statistic: {f_stat:.3f}")
    print(f"P-value: {p_value:.4f}")
    
    if p_value < 0.05:
        print("✅ Las diferencias entre grupos SON significativas")
    else:
        print("❌ No hay diferencias significativas entre grupos")
    
    # Visualización
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Boxplot
    df.boxplot(column=num_var, by=cat_var, ax=axes[0])
    axes[0].set_title(f'Distribución de {num_var} por {cat_var}\n(F = {f_stat:.2f}, p = {p_value:.4f})')
    axes[0].set_xlabel(cat_var)
    axes[0].set_ylabel(num_var)
    plt.sca(axes[0])
    plt.xticks(rotation=45)
    
    # Violin plot (muestra distribución completa)
    sns.violinplot(data=df, x=cat_var, y=num_var, ax=axes[1])
    axes[1].set_title(f'Violin Plot: {num_var} por {cat_var}')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return stats_grupos, f_stat, p_value

# Uso: Análisis de tiempo de sesión por dispositivo
stats_tiempo, f_stat, p_val = analizar_relacion_mixta(
    df, 'dispositivo', 'tiempo_sesion'
)
```

---

## 7. Caso práctico: E-commerce completo

**Contexto:** Dataset de 100,000 sesiones de usuario con métricas de comportamiento y conversión.

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# 1. CARGAR DATOS
df = pd.read_csv('sesiones_ecommerce.csv')
print(f"Dataset: {df.shape[0]} sesiones, {df.shape[1]} variables")

# 2. ANÁLISIS DE CORRELACIONES NUMÉRICAS
print("\n" + "="*60)
print("PASO 1: CORRELACIONES ENTRE MÉTRICAS NUMÉRICAS")
print("="*60)

metricas = ['tiempo_sesion', 'paginas_vistas', 'ratio_rebote', 'tasa_conversion']
matriz_corr = matriz_correlacion_completa(df[metricas])

# Correlaciones con variable objetivo
correlaciones_conversion = df[metricas].corrwith(df['tasa_conversion']).sort_values(ascending=False)
print("\nCorrelaciones con CONVERSIÓN:")
print(correlaciones_conversion)

# 3. ANÁLISIS DETALLADO: Tiempo vs Conversión
print("\n" + "="*60)
print("PASO 2: ANÁLISIS DETALLADO - Tiempo vs Conversión")
print("="*60)

resultado_tiempo = analizar_correlacion_pareada(
    df, 'tiempo_sesion', 'tasa_conversion'
)

# 4. ANÁLISIS CATEGÓRICO: Dispositivo vs Conversión
print("\n" + "="*60)
print("PASO 3: ANÁLISIS CATEGÓRICO - Dispositivo vs Conversión")
print("="*60)

df['dispositivo'] = df['user_agent'].apply(
    lambda x: 'Mobile' if 'Mobile' in str(x) else 'Desktop'
)
tabla_disp, chi2_disp, p_disp, cramers_disp = analizar_relaciones_categoricas(
    df, 'dispositivo', 'convertido', normalizar=True
)

# 5. ANÁLISIS MIXTO: Tiempo de sesión por dispositivo
print("\n" + "="*60)
print("PASO 4: ANÁLISIS MIXTO - Tiempo por Dispositivo")
print("="*60)

stats_tiempo_disp, f_stat, p_val = analizar_relacion_mixta(
    df, 'dispositivo', 'tiempo_sesion'
)

# 6. RESUMEN DE HALLAZGOS
print("\n" + "="*60)
print("HALLAZGOS CLAVE")
print("="*60)
print("🔍 Correlación fuerte: tiempo_sesion vs paginas_vistas (r = 0.82)")
print("📱 Relación significativa: Mobile tiene 35% menos conversión que Desktop")
print("⏱️  Diferencia marcada: Usuarios Desktop permanecen 2.3x más tiempo")
print("🎯 Insight accionable: Mejorar experiencia Mobile podría aumentar conversiones 25%")
```

**Resultados cuantificados del caso:**
- **Correlación más fuerte:** tiempo_sesion vs paginas_vistas (r=0.82, p<0.001)
- **Diferencia Mobile vs Desktop:** 35% menos conversión en Mobile (χ²=450, p<0.001)
- **Tiempo de sesión:** Desktop 2.3x mayor que Mobile (F=380, p<0.001)
- **ROI potencial:** Mejora en Mobile = +25% conversiones = +€500K/año

---

## 8. Técnicas avanzadas

### **Matriz de correlación con significancia**

```python
def matriz_correlacion_con_significancia(df):
    """Matriz de correlación que incluye significancia estadística"""
    numeric_df = df.select_dtypes(include=[np.number])
    columns = numeric_df.columns
    n = len(columns)
    
    # Crear matrices
    corr_matrix = np.zeros((n, n))
    p_value_matrix = np.zeros((n, n))
    
    # Calcular correlaciones y p-values
    for i in range(n):
        for j in range(n):
            if i == j:
                corr_matrix[i, j] = 1.0
                p_value_matrix[i, j] = 0.0
            else:
                corr, p_value = stats.pearsonr(
                    numeric_df.iloc[:, i],
                    numeric_df.iloc[:, j]
                )
                corr_matrix[i, j] = corr
                p_value_matrix[i, j] = p_value
    
    # Crear matriz con asteriscos de significancia
    significance_df = pd.DataFrame('', index=columns, columns=columns)
    for i in range(n):
        for j in range(n):
            corr_val = corr_matrix[i, j]
            p_val = p_value_matrix[i, j]
            
            if p_val < 0.001:
                significance_df.iloc[i, j] = f"{corr_val:.2f}***"
            elif p_val < 0.01:
                significance_df.iloc[i, j] = f"{corr_val:.2f}**"
            elif p_val < 0.05:
                significance_df.iloc[i, j] = f"{corr_val:.2f}*"
            else:
                significance_df.iloc[i, j] = f"{corr_val:.2f}"
    
    print("\n=== MATRIZ DE CORRELACIÓN CON SIGNIFICANCIA ===")
    print("*** p < 0.001, ** p < 0.01, * p < 0.05")
    print(significance_df)
    
    return significance_df

# Uso
matriz_sig = matriz_correlacion_con_significancia(df)
```

### **Correlación parcial (técnica avanzada)**

```python
def correlacion_parcial(df, var1, var2, control_vars):
    """Calcula correlación parcial controlando por otras variables"""
    from sklearn.linear_model import LinearRegression
    
    # Variables limpias
    data = df[[var1, var2] + control_vars].dropna()
    X_control = data[control_vars]
    y1 = data[var1]
    y2 = data[var2]
    
    # Modelos para eliminar efecto de variables de control
    model1 = LinearRegression().fit(X_control, y1)
    model2 = LinearRegression().fit(X_control, y2)
    
    # Residuales (efecto puro de var1 y var2)
    resid1 = y1 - model1.predict(X_control)
    resid2 = y2 - model2.predict(X_control)
    
    # Correlación entre residuales = correlación parcial
    corr_parcial, p_value = stats.pearsonr(resid1, resid2)
    
    # Comparar con correlación simple
    corr_simple, p_simple = stats.pearsonr(data[var1], data[var2])
    
    print(f"\n=== CORRELACIÓN PARCIAL: {var1} vs {var2} ===")
    print(f"Controlando por: {control_vars}")
    print(f"Correlación simple:  {corr_simple:.3f} (p = {p_simple:.4f})")
    print(f"Correlación parcial: {corr_parcial:.3f} (p = {p_value:.4f})")
    print(f"Diferencia: {corr_parcial - corr_simple:+.3f}")
    
    if abs(corr_parcial) < abs(corr_simple):
        print("\n💡 La variable de control explica parte de la relación")
    else:
        print("\n💡 La relación persiste incluso controlando por otras variables")
    
    return corr_parcial, p_value

# Ejemplo: ¿La relación edad-ingresos es real o solo por educación?
corr_parcial, p_val = correlacion_parcial(
    df, 'edad', 'ingresos', ['nivel_educativo']
)
```

---

## 9. Correlación ≠ Causalidad: Framework de evaluación

### **Casos clásicos de correlación sin causalidad**

| Correlación | Explicación | Tercer factor |
|-------------|-------------|---------------|
| **Helados vs Ahogamientos** | Ambos suben en verano | Temperatura |
| **Piratas vs Calentamiento global** | Menos piratas, más calentamiento | Tiempo (progreso) |
| **Nicolas Cage vs Ahogamientos** | Coincidencia pura | Ninguno (aleatorio) |

### **Framework para evaluar causalidad potencial**

```python
def verificar_causalidad_potencial(df, var1, var2, variables_confusoras=None):
    """Framework para evaluar causalidad potencial (NO es prueba)"""
    print(f"\n=== EVALUANDO CAUSALIDAD POTENCIAL ===")
    print(f"¿{var1} → {var2}?")
    
    # 1. Fuerza de correlación
    corr, p_val = stats.pearsonr(df[var1].dropna(), df[var2].dropna())
    print(f"\n1. Correlación: r = {corr:.3f} (p = {p_val:.4f})")
    
    # 2. Temporalidad
    if 'fecha' in df.columns:
        print("2. Temporalidad: ✅ Verificable con datos temporales")
    else:
        print("2. Temporalidad: ⚠️  No verificable sin datos temporales")
    
    # 3. Variables confusoras
    if variables_confusoras:
        print(f"3. Variables confusoras: {variables_confusoras}")
        print("   → Analizar correlación parcial")
    else:
        print("3. Variables confusoras: ⚠️  No especificadas")
    
    # 4. Mecanismo plausible
    print("4. Mecanismo: ❓ ¿Hay explicación teórica plausible?")
    
    # 5. Recomendación
    print("\n=== RECOMENDACIÓN ===")
    if abs(corr) > 0.5 and p_val < 0.05:
        print("🎯 Vale la pena investigar causalidad con:")
        print("   • Experimento A/B controlado")
        print("   • Análisis de series temporales")
        print("   • Métodos causales avanzados (regresión discontinua, etc.)")
    else:
        print("⚠️  Probablemente NO es causalidad, solo correlación espuria")

# Uso
verificar_causalidad_potencial(
    df, 'gasto_publicidad', 'ventas',
    ['estacion', 'competencia', 'precio']
)
```

> **Regla de oro:** Para afirmar causalidad necesitas:
> 1. Experimento controlado (A/B test) O
> 2. Métodos causales avanzados O
> 3. Mecanismo físico/biológico demostrado

---

## 10. Checklist y guía de decisión

### **Cuándo usar cada técnica**

| Escenario | Técnica recomendada | Output clave |
|-----------|---------------------|--------------|
| **2 variables numéricas** | Correlación Pearson/Spearman | r, p-value |
| **2 variables categóricas** | Tabla cruzada + Chi-cuadrado | χ², Cramér's V |
| **Numérica vs Categórica** | ANOVA + Boxplots | F-statistic, visualización |
| **Múltiples variables** | Matriz correlación + Pairplot | Mapa completo |
| **Controlar variables** | Correlación parcial | r parcial |

### **Checklist de análisis de relaciones**

- [ ] Variables numéricas: Correlación calculada (Pearson/Spearman)
- [ ] P-values verificados (< 0.05 para significancia)
- [ ] Matriz de correlación visualizada (heatmap)
- [ ] Outliers identificados y evaluados
- [ ] Variables categóricas: Tablas cruzadas creadas
- [ ] Test Chi-cuadrado ejecutado si aplica
- [ ] Cramér's V calculado para fuerza de asociación
- [ ] Relaciones mixtas: ANOVA realizado
- [ ] Visualizaciones apropiadas generadas
- [ ] Contexto de negocio considerado
- [ ] Causalidad NO asumida sin evidencia experimental

---

## 11. Resumen

**Tipos de relaciones por variables:**
- ✅ **Numérica-Numérica** → Correlación (Pearson r, Spearman ρ, Kendall τ)
- ✅ **Categórica-Categórica** → Tabla cruzada + Chi-cuadrado (χ²) + Cramér's V
- ✅ **Numérica-Categórica** → ANOVA (F) + Boxplot

**Interpretación de r (correlación):**
- |r| > 0.7 → Fuerte
- |r| = 0.5-0.7 → Moderada
- |r| = 0.3-0.5 → Débil
- |r| < 0.3 → Muy débil

**Interpretación de Cramér's V:**
- V > 0.5 → Asociación fuerte
- V = 0.3-0.5 → Moderada
- V = 0.1-0.3 → Débil
- V < 0.1 → Muy débil

**Aplicaciones empresariales:**
- **Marketing:** Gasto publicidad vs ventas por canal
- **Producto:** Features usage vs retention
- **Operaciones:** Fallos por tipo de equipo y turno
- **RRHH:** Satisfacción empleados vs productividad

**La advertencia más importante:**
```
CORRELACIÓN ≠ CAUSALIDAD

Helados ↔ Ahogamientos (r=0.9)
Pero helados NO causan ahogamientos
(Ambos suben con la temperatura)
```

> **Conclusión:** Dominar las relaciones entre variables es como tener un mapa del tesoro de tu negocio. Te muestra dónde encontrar insights valiosos y te previene de conclusiones falsas. La clave: siempre visualiza, siempre valida significancia, nunca asumas causalidad.

---

## 12. Referencias

### Vídeos
- [Correlation vs Causation](https://youtu.be/example1) - Ejemplos visuales
- [Understanding Chi-Square](https://youtu.be/example2) - Test de independencia
- [Spurious Correlations](https://youtu.be/example3) - Correlaciones absurdas

### Lecturas
- ["Naked Statistics" - Wheelan](https://example.com) - Intuición sobre correlación
- [Correlation in Python](https://realpython.com/numpy-scipy-pandas-correlation-python/)
- [Spurious Correlations Website](https://tylervigen.com/spurious-correlations) - ¡Divertido!

### Herramientas
- [Pandas corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html)
- [SciPy stats](https://docs.scipy.org/doc/scipy/reference/stats.html)
- [pandas-profiling](https://github.com/pandas-profiling/pandas-profiling) - Análisis automático